<a href="https://colab.research.google.com/github/7PAM2015-0509-2023/Spaceship-Titanic/blob/main/Kaggle_Challenge_Spaceship_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Anzar Naseer

In [ ]:
# Import Libraries and Load Data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE

In [ ]:
# Load the dataset
train_data = 'https://raw.githubusercontent.com/7PAM2015-0509-2023/Spaceship-Titanic/main/train.csv?token=GHSAT0AAAAAACUMVGDA5ZJG56COCUEV4ZPWZUFEJPQ'
test_data = 'https://raw.githubusercontent.com/7PAM2015-0509-2023/Spaceship-Titanic/main/test.csv?token=GHSAT0AAAAAACUMVGDASW5CK4QG56WCKYWOZUFEI5A'
train_df = pd.read_csv(train_data)
test_df = pd.read_csv(test_data)